# Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

## import packages

In [1]:
import os
import requests 
from pprint import pprint

from tqdm.auto import tqdm
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

# Load environment variables from the .env file
load_dotenv()

True

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [2]:
es_client = Elasticsearch('http://localhost:9200') 
es_client_info = es_client.info()

pprint(es_client_info['version']['build_hash'])

'42f05b9372a9a4a470db3b52817899b99a76ee73'


### A1: `42f05b9372a9a4a470db3b52817899b99a76ee73`

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### A2: `index`

## Q3. Searching

Now let's search in our index. 

For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result? 

Use only `question` and `text` fields and give `question` a boost of 4

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

### the query string

In [5]:
query = 'How do I execute a command in a running docker container?'

In [6]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    # result_docs.append(hit['_source'])
    pprint(hit["_score"])

# pprint(result_docs)

83.765434
76.374596
51.012703
49.506863
45.826126


### A3: `84.050095`

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [7]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", ],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    print(f"scores: {hit['_score']}")
    
print()
for result in result_docs:
    print(f"question: {result['question']}") #\nanswer: {result['text']}\n\n

scores: 83.765434
scores: 51.012703
scores: 49.506863

question: How do I debug a docker container?
question: How do I copy files from my local machine to docker container?
question: How do I copy files from a different folder into docker container’s working directory?


### A4: `How do I copy files from a different folder into docker container’s working directory?`

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. Let's use these templates:


```python
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

Separate context entries by two linebreaks (`\n\n`)

Put the data from Q4 there. 

What's the length of the result? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [8]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [9]:
qn5_result = build_prompt(query, result_docs)

print(f'qn5_result length: {len(qn5_result)}')

# print()
# pprint(qn5_result)

qn5_result length: 1462


### A5: `1462`

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [10]:
# !pip install tiktoken

In [11]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [12]:
len(encoding.encode(qn5_result))


322

### A6: `322`

## Q7 Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

### using OpenAI


In [13]:
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("openai_API_KEY"),
)

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

  0%|          | 0/948 [00:00<?, ?it/s]

In [15]:
openAI_answer = rag(query=query)
pprint(openAI_answer)

('To execute a command in a running Docker container, follow these steps:\n'
 '\n'
 '1. First, find the container ID of the running container by using the '
 'command:\n'
 '   ```\n'
 '   docker ps\n'
 '   ```\n'
 '\n'
 '2. Once you have the container ID, use the following command to execute a '
 'command within the container:\n'
 '   ```\n'
 '   docker exec -it <container-id> <command>\n'
 '   ```\n'
 '\n'
 'For example, if you want to start a bash session inside the container, you '
 'would use:\n'
 '   ```\n'
 '   docker exec -it <container-id> bash\n'
 '   ```')


### A7 bonus: see previous cell


## Q8 Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

In [16]:
def estimate_cost(requests, input_text, output_text):
    # Initialize tiktoken encoders
    enc = tiktoken.encoding_for_model("gpt-4o")

    # Calculate tokens for a single request
    input_tokens_per_request = len(enc.encode(input_text))
    output_tokens_per_request = len(enc.encode(output_text))
    
    # Prices
    price_per_1k_input_tokens = 0.005
    price_per_1k_output_tokens = 0.015
    
    # Calculate the total number of tokens for all requests
    total_input_tokens = input_tokens_per_request * requests
    total_output_tokens = output_tokens_per_request * requests
    
    # Suppose that on average per request we send 150 tokens and receive back 250 tokens.
    # total_input_tokens = 150 * requests
    # total_output_tokens = 250 * requests

    # Calculate the cost
    input_cost = (total_input_tokens / 1000) * price_per_1k_input_tokens
    output_cost = (total_output_tokens / 1000) * price_per_1k_output_tokens
    
    total_cost = input_cost + output_cost
    
    return total_cost

In [17]:
# Example usage from chatGPT
requests = 1000
estimated_cost = estimate_cost(requests, query, openAI_answer)
print(f"The estimated cost for {requests} requests is: ${estimated_cost:.4f}")


The estimated cost for 1000 requests is: $1.7400


### A8 bonus

using input=150, output=250

`The estimated cost for 1000 requests is: $4.5000 `

---

using actual query and LLM's response. It's cheaper?

`The estimated cost for 1000 requests is: $1.7400 `
